In [2]:
import pandas as pd
import plotly.express as px

In [44]:
# load data
df = pd.read_csv('../data/UAB_STREP.csv')

# rename columns
df.columns = [_.lower().replace(' ', '_') for _ in df.columns.to_list()]

# set fiscal year to string
df.fiscal_year = df.fiscal_year.astype(str)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455807 entries, 0 to 455806
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   fiscal_year         455807 non-null  object 
 1   transaction_number  455807 non-null  object 
 2   agency_name         455807 non-null  object 
 3   source              455807 non-null  object 
 4   category            455807 non-null  object 
 5   sub_category        455807 non-null  object 
 6   payee               455807 non-null  object 
 7   payment_date        455807 non-null  object 
 8   amount              455807 non-null  float64
dtypes: float64(1), object(8)
memory usage: 31.3+ MB


In [45]:
df.describe(include='all')

,fiscal_year,transaction_number,agency_name,source,category,sub_category,payee,payment_date,amount
count,455807,455807,455807,455807,455807,455807,455807,455807,4.558070e+05
unique,1,455807,1,1,12,60,34507,234,NaN
top,2021,20210226-1093512,UAB,UAB,SALARY WAGES,SALARY-PROFESSIONAL NON-FACULTY,FISHER SCIENTIFIC,30-APR-2021,NaN
freq,455807,1,455807,455807,389547,193559,788,21169,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.292900e+03
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.969370e+04
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.952164e+04
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.101195e+03
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.966280e+03
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.026500e+03


In [46]:
# set transaction number as index
df.set_index(keys='transaction_number', inplace=True)

# remove agency name and source given there is no variation
cols = ['agency_name', 'source']
df.drop(columns=cols, axis=1, inplace=True)

In [47]:
# counts and props by category
# majortity are salary wages
pd.concat([df.category.value_counts(), df.category.value_counts() / len(df)], axis=1)

,category,category
SALARY WAGES,389547,0.854631
SUPPL/MAT/NONCAP EQU,26117,0.057298
SERVICES,17839,0.039137
OTHER EXPENSES,10402,0.022821
REPAIRS & MAINT,4998,0.010965
TRAVEL & EMPL TRAING,4047,0.008879
NET INV IN PLANT,1179,0.002587
TELECOMMUNICATIONS,565,0.001240
BUSINESS MEALS/ENT,524,0.001150
BENEFITS,388,0.000851


In [48]:
# limit to just salary wages
df = df[df.category == 'SALARY WAGES']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 389547 entries, 20201002-1000025 to 20210831-1113205
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   fiscal_year   389547 non-null  object 
 1   category      389547 non-null  object 
 2   sub_category  389547 non-null  object 
 3   payee         389547 non-null  object 
 4   payment_date  389547 non-null  object 
 5   amount        389547 non-null  float64
dtypes: float64(1), object(5)
memory usage: 20.8+ MB


In [49]:
# subcategory counts and props
pd.concat([df.sub_category.value_counts(), df.sub_category.value_counts() / len(df)], axis=1)

,sub_category,sub_category
SALARY-PROFESSIONAL NON-FACULTY,193559,0.496882
SALARY-TECHNICAL/PARA-PROFESSIONAL,58267,0.149576
SALARY-SKILLED CRAFT/SUPPORT SERVICES,54063,0.138784
SALARY-SECRETARIAL/CLERICAL,41895,0.107548
SALARY -INSTR/FELL/FACULTY,34331,0.088131
"SALARY-EXEC, ADMIN, MANAGEMENT",7427,0.019066
PROFESSIONAL SERVICES,5,0.000013


In [71]:
pd.set_option('display.float_format', '{:.2f}'.format)

df.groupby(['sub_category']).agg(
    total_amount=('amount', sum),
    records=('amount', len),
    avg_amount = ('amount', 'mean'),
    med_amount = ('amount', 'median')).sort_values(by='med_amount', ascending=False)

,total_amount,records,avg_amount,med_amount
sub_category,,,,
"SALARY-EXEC, ADMIN, MANAGEMENT",81316476.04,7427.00,10948.76,9644.04
PROFESSIONAL SERVICES,25401.80,5.00,5080.36,7096.77
SALARY -INSTR/FELL/FACULTY,274079393.30,34331.00,7983.44,7074.72
SALARY-PROFESSIONAL NON-FACULTY,666423073.85,193559.00,3443.00,2654.33
SALARY-TECHNICAL/PARA-PROFESSIONAL,103468956.43,58267.00,1775.77,1557.97
SALARY-SECRETARIAL/CLERICAL,58758347.82,41895.00,1402.51,1358.41
SALARY-SKILLED CRAFT/SUPPORT SERVICES,67142435.39,54063.00,1241.93,1101.60
